In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL_SPARK").getOrCreate()

In [2]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# EXTRACT

In [3]:
file_path = r"C:\Users\Public\sales_csv\Sales_June_2019.csv"
sales_csv = spark.read.csv(file_path , header = True , inferSchema = True)
sales_csv.show()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/C:/Users/Public/sales_csv/Sales_June_2019.csv.

In [ ]:
sales_csv.count()

# TRANSFORM

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, month, dayofmonth, lit, split

# Assuming you have a SparkSession named spark
# sales_csv = spark.read...  # Include your method for reading data

sales_csv = sales_csv.na.drop(how='all')

sales_csv = sales_csv.withColumnRenamed("Order ID", "order_id") \
        .withColumnRenamed("Product", "product") \
        .withColumnRenamed("Quantity Ordered", "quantity_ordered") \
        .withColumnRenamed("Price Each", "price_each") \
        .withColumnRenamed("Order Date", "order_date") \
        .withColumnRenamed("Purchase Address", "address")

sales_csv = sales_csv.withColumn("order_date", to_date(col("order_date"), "M/d/yyyy"))

# Extracting year, month, and day from the order date
sales_csv = sales_csv.withColumn("order_year", year("order_date"))
sales_csv = sales_csv.withColumn("order_month", month("order_date"))
sales_csv = sales_csv.withColumn("order_day", dayofmonth("order_date"))

address_split =  split(sales_csv["address"], ", ")
sales_csv = sales_csv.withColumn("street", address_split.getItem(0))
sales_csv = sales_csv.withColumn("city", address_split.getItem(1))
sales_csv = sales_csv.withColumn("state_zip", address_split.getItem(2))

zip_split = split(sales_csv["state_zip"], " ")
sales_csv = sales_csv.withColumn("state", zip_split.getItem(0))
sales_csv = sales_csv.withColumn("zip", zip_split.getItem(1))

# Adding the 'country' column
sales_csv = sales_csv.withColumn("country",  lit("United States"))

# Adding the 'Total_Amount' column
sales_csv = sales_csv.withColumn("Total_Amount",col("quantity_ordered") * col("price_each") )

# Dropping unnecessary columns
sales_csv = sales_csv.drop("address", "state_zip")

# Selecting specific columns
sales_csv = sales_csv.select('order_id', 'order_year', 'order_month', 'order_day',
                             'product', 'quantity_ordered', 'price_each',"Total_Amount",
                             'street', 'city', 'state', 'zip', 'country')

# drop null values
sales_csv.na.drop(how='any')
    # Remove duplicates
sales_csv = sales_csv.dropDuplicates()

sales_csv.columns


NameError: name 'sales_csv' is not defined

In [8]:
sales_csv.show()


NameError: name 'sales_csv' is not defined

In [9]:
sales_csv.count()

13538

In [2]:
import os

output_csv_path = r"C:\Users\Public"
output_name = "sales_csv_spark"

sales_csv.write.csv(os.path.join(output_csv_path,output_name) ,header = True , mode = "Overwrite")

NameError: name 'sales_csv' is not defined